## Demo: Stage1 processing for mooring data

**Stage 1 Overview**: This is the first processing stage that converts raw instrument files into standardized CF-compliant NetCDF format. It handles multiple instrument types using the `ctd_tools` library for reading and the oceanarray framework for metadata management.

### What Stage1 Does:
- **File Conversion**: Reads raw instrument files (SeaBird, RBR, Nortek, etc.) and converts to NetCDF
- **Standardization**: Applies CF conventions for variable names, units, and metadata
- **Format Preservation**: Preserves original data values without modification or filtering
- **Metadata Enrichment**: Adds deployment information from YAML configuration files
- **Organization**: Outputs files organized by instrument type in the processed directory

### Input Files:
- Raw instrument data files (various formats: `.cnv`, `.rsk`, `.dat`, `.mat`)
- YAML configuration files specifying mooring and instrument metadata

### Output Files:
- Standardized NetCDF files: `{mooring}_{serial}_raw.nc`
- Processing log files for debugging and quality assurance

### Processing Flow:
1. Raw files → ctd_tools readers → xarray.Dataset
2. Apply CF conventions and metadata from YAML
3. Preserve all original data values unchanged
4. Save as NetCDF with standardized naming

**Note**: Stage1 focuses purely on format conversion with no data modification. All processing (filtering, clock corrections, quality control, trimming) happens in Stage2 and subsequent stages.

This notebook demonstrates the usage of the refactored `oceanarray.stage1` module.

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from pathlib import Path

# Import the refactored stage1 processing module
from oceanarray.stage1 import MooringProcessor, process_multiple_moorings, stage1_mooring

# For testing individual readers (optional)
from ctd_tools.readers import NortekAsciiReader, AdcpMatlabReader
from ctd_tools.plotters import TimeSeriesPlotter

## Configuration

Set up the base directory and mooring lists for processing.

In [ ]:
# Base directory containing the mooring data
basedir = '/Users/eddifying/Dropbox/data/ifmro_mixsed/ds_data_eleanor/'

# Define mooring lists
all_moorings = ['ds2_X_2012', 'ds2_X_2017', 'ds2_X_2018',
                'ds8_1_2012', 'ds9_1_2012', 'ds10_1_2012', 'ds11_1_2012', 'ds12_1_2012',
                'ds13_1_2012', 'ds14_1_2012', 'ds15_1_2012', 'ds16_1_2012', 'ds17_1_2012',
                'ds19_1_2012', 'ds18_1_2012', 'ds28_1_2017',
                'dsA_1_2018', 'dsB_1_2018', 'dsC_1_2018', 'dsD_1_2018', 'dsE_1_2018', 'dsF_1_2018',
                'dsM1_1_2017', 'dsM2_1_2017', 'dsM3_1_2017', 'dsM4_1_2017', 'dsM5_1_2017']

# Subset for testing
test_moorings_2018 = ['dsA_1_2018', 'dsB_1_2018', 'dsC_1_2018', 'dsD_1_2018', 'dsE_1_2018', 'dsF_1_2018']
single_test = ['dsB_1_2018']

# Choose which set to process
moorlist = single_test

print(f"Base directory: {basedir}")
print(f"Processing {len(moorlist)} moorings: {moorlist}")

## Processing Moorings

### Option 1: Using the new class-based approach

In [ ]:
# Initialize the processor
processor = MooringProcessor(basedir)

# Process each mooring individually with detailed output
results = {}
for mooring_name in moorlist:
    print(f"\n{'='*60}")
    print(f"Processing mooring: {mooring_name}")
    print(f"{'='*60}")

    success = processor.process_mooring(mooring_name)
    results[mooring_name] = success

    status = "✅ SUCCESS" if success else "❌ FAILED"
    print(f"\nResult for {mooring_name}: {status}")

# Print final summary
print(f"\n{'='*60}")
print("FINAL PROCESSING SUMMARY")
print(f"{'='*60}")
successful = sum(results.values())
total = len(results)
print(f"Successfully processed: {successful}/{total} moorings")

for mooring, success in results.items():
    status = "✅" if success else "❌"
    print(f"{status} {mooring}")

### Option 2: Using the convenience function for batch processing

In [ ]:
# Alternative: Use the batch processing function
# Uncomment to use this approach instead

if 0<0:
    results = process_multiple_moorings(moorlist, basedir)

    print(f"\n{'='*50}")
    print("BATCH PROCESSING RESULTS")
    print(f"{'='*50}")
    for mooring, success in results.items():
        status = "SUCCESS" if success else "FAILED"
        print(f"{mooring}: {status}")

### Option 3: Using the backwards-compatible function

In [ ]:
# Alternative: Use the original function signature for backwards compatibility
# Uncomment to use this approach instead

if 0:
    for mooring_name in moorlist:
        print(f"Processing {mooring_name}...")
        success = stage1_mooring(mooring_name, basedir=basedir)
        status = "SUCCESS" if success else "FAILED"
        print(f"{mooring_name}: {status}\n")

## Testing Individual Readers

Test reading specific instrument files directly to debug issues.

In [ ]:
# Test reading a Nortek AquaDopp file directly
try:
    rawdir = Path(basedir) / 'moor/raw/msm76_2018/'
    instrument = 'aquadopp'
    data_dir = rawdir / instrument
    fname = 'DSC18_477102.dat'
    filename = data_dir / fname
    header_file = data_dir / (fname[:-4] + '.hdr')

    print(f"Data file: {filename}")
    print(f"Header file: {header_file}")
    print(f"Files exist: data={filename.exists()}, header={header_file.exists()}")

    if filename.exists() and header_file.exists():
        reader = NortekAsciiReader(str(filename), header_file_path=str(header_file))
        dataset = reader.get_data()

        print(f"\nDataset loaded successfully!")
        print(f"Variables: {list(dataset.data_vars)}")
        print(f"Time range: {dataset.time.min().values} to {dataset.time.max().values}")

        # Plot if east_velocity exists
        if 'east_velocity' in dataset.data_vars:
            plotter = TimeSeriesPlotter(dataset)
            plotter.plot(parameter_name='east_velocity')
            plt.title(f'East Velocity - {fname}')
            plt.show()

        # Display dataset info
        display(dataset)
    else:
        print("Files not found - skipping test")

except Exception as e:
    print(f"Error testing Nortek reader: {e}")

In [ ]:
# Test reading an ADCP MATLAB file directly
try:
    rawdir = Path(basedir) / 'moor/raw/msm76_2018/'
    instrument = 'adcp'
    data_dir = rawdir / instrument
    fname = 'DS0218_RDI_000_24289.mat'
    filename = data_dir / fname

    print(f"ADCP file: {filename}")
    print(f"File exists: {filename.exists()}")

    if filename.exists():
        reader = AdcpMatlabReader(str(filename))
        dataset = reader.get_data()

        print(f"\nADCP Dataset loaded successfully!")
        print(f"Variables: {list(dataset.data_vars)}")
        print(f"Time range: {dataset.time.min().values} to {dataset.time.max().values}")

        # Plot pressure if it exists
        if 'pressure' in dataset.data_vars:
            plt.figure(figsize=(12, 4))
            plt.plot(dataset.time, dataset.pressure)
            plt.title(f'Pressure - {fname}')
            plt.xlabel('Time')
            plt.ylabel('Pressure')
            plt.grid(True)
            plt.show()

        # Display dataset info
        display(dataset)
    else:
        print("File not found - skipping test")

except Exception as e:
    print(f"Error testing ADCP reader: {e}")

## Analyzing Processed Results

Load and visualize the processed NetCDF files.

In [ ]:
# Analyze processed files
def analyze_processed_mooring(mooring_name, basedir, instrument_type='microcat'):
    """Analyze processed files for a specific mooring and instrument type."""
    proc_dir = Path(basedir) / 'moor/proc' / mooring_name / instrument_type
    fig_dir = proc_dir

    if not proc_dir.exists():
        print(f"Directory not found: {proc_dir}")
        return

    print(f"Analyzing processed files in: {proc_dir}")

    # Find all NetCDF files
    nc_files = list(proc_dir.glob('*raw.nc'))
    if not nc_files:
        print("No NetCDF files found")
        return

    print(f"Found {len(nc_files)} NetCDF files:")

    for file_path in nc_files:
        try:
            print(f"\n📄 Loading: {file_path.name}")

            with xr.open_dataset(file_path) as dataset:
                print(f"   Variables: {list(dataset.data_vars)}")
                print(f"   Attributes: {list(dataset.attrs.keys())}")

                # Extract key metadata
                mooring_name_attr = dataset.attrs.get('mooring_name', 'Unknown')
                instrument_name = dataset.get('instrument', 'Unknown').values if 'instrument' in dataset else 'Unknown'
                serial_number = dataset.get('serial_number', 0).values if 'serial_number' in dataset else 0

                print(f"   Mooring: {mooring_name_attr}")
                print(f"   Instrument: {instrument_name}")
                print(f"   Serial: {serial_number}")

                # Plot temperature if available
                if 'temperature' in dataset.data_vars:
                    fig, ax = plt.subplots(figsize=(12, 4))
                    ax.plot(dataset['time'], dataset['temperature'], 'b-', linewidth=0.5)
                    ax.set_ylabel('Temperature (°C)')
                    ax.set_title(f'{mooring_name_attr}: {instrument_name} {serial_number} - Temperature')
                    ax.set_xlabel('Time')
                    ax.grid(True, alpha=0.3)

                    # Add stats
                    temp_mean = dataset['temperature'].mean().values
                    temp_std = dataset['temperature'].std().values
                    ax.text(0.02, 0.98, f'Mean: {temp_mean:.2f}°C\nStd: {temp_std:.2f}°C',
                           transform=ax.transAxes, verticalalignment='top',
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

                    plt.tight_layout()
                    plt.show()

                    # Save plot
                    plot_name = f"{mooring_name_attr}_{instrument_name}_{serial_number}_temperature_raw.png"
                    fig.savefig(fig_dir / plot_name, dpi=150, bbox_inches='tight')
                    print(f"   📊 Plot saved: {plot_name}")

        except Exception as e:
            print(f"   ❌ Error loading {file_path.name}: {e}")

# Example usage
if results and any(results.values()):
    # Analyze the first successfully processed mooring
    successful_mooring = next(mooring for mooring, success in results.items() if success)
    print(f"\nAnalyzing successful mooring: {successful_mooring}")
    analyze_processed_mooring(successful_mooring, basedir, 'microcat')
else:
    print("No successfully processed moorings to analyze")